# Use PREDICT for MLFLOW packaged models - Registered on AML

[Source](https://learn.microsoft.com/en-us/azure/synapse-analytics/machine-learning/tutorial-score-model-predict-spark-pool#use-predict-for-mlflow-packaged-models)


##
## Import Librairies


In [1]:
#Import libraries
from pyspark.sql.functions import col, pandas_udf,udf,lit
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication
import azure.synapse.ml.predict as pcontext
import azure.synapse.ml.predict.utils._logger as synapse_predict_logger

StatementMeta(sparkezi1n72, 82, 1, Finished, Available)

## Set some parameters

In [25]:
#Set input data path
DATA_FILE = "abfss://<filesystemname>@<account name>.dfs.core.windows.net/<file path>"

#Set model URI
#Set AML URI, if trained model is registered in AML

AML_MODEL_URI = "<aml model uri> #In URI ":x" signifies model version in AML. You can   choose which model version you want to run. If ":x" is not provided then by default   latest version will be picked.


#Define model return type
RETURN_TYPES = "INT" # for ex: int, float etc. PySpark data types are supported

#Define model runtime. This supports only mlflow
RUNTIME = "mlflow"

StatementMeta(sparkezi1n72, 82, 25, Finished, Available)

## Authenticate to AML through linked service

In [26]:
#AML workspace authentication using linked service
from notebookutils.mssparkutils import azureML
ws = azureML.getWorkspace("AzureMLService1") #   "<linked_service_name>" is the linked service name, not AML workspace name. Also, linked   service supports MSI and service principal both

StatementMeta(sparkezi1n72, 82, 26, Finished, Available)

## Enable PREDICT in spark session

In [46]:
#Enable SynapseML predict
spark.conf.set("spark.synapse.ml.predict.enabled","true")

StatementMeta(sparkezi1n72, 82, 46, Finished, Available)

## Bind model in spark session

In [28]:
#Bind model within Spark session
model = pcontext.bind_model(
    return_types=RETURN_TYPES, 
    runtime=RUNTIME, 
    model_alias="mlflow_credit_defaults", #This alias will be used in PREDICT call to refer  this   model
    model_uri=AML_MODEL_URI, #In case of AML, it will be AML_MODEL_URI
    aml_workspace=ws #This is only for AML. In case of ADLS, this parameter can be removed
    ).register()

StatementMeta(sparkezi1n72, 82, 28, Finished, Available)

##

## Read data from ADLS

In [31]:
df = spark.read \
   .format("csv") \
   .option("header", "true") \
   .csv(DATA_FILE,
    inferSchema=True)
df.createOrReplaceTempView('test_data')

StatementMeta(sparkezi1n72, 82, 31, Finished, Available)

In [32]:
df.show(5)

StatementMeta(sparkezi1n72, 82, 32, Finished, Available)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+
|   640000|  1|        1|       2| 36|    0|    0|    0|    0|    0|    0|   192325|    68891|    71564|    32968|    37541|    42064|    4000|    4000|    3000|    5000|    5000|    3000|
|   100000|  1|        2|       2| 44|    0|    0|    0|   -1|   -1|    0|    20439|     6882|     2456|     1537|     1642|      110|    1068|    1007|    1741|    1646|     100|       0|
|    70000|  2|        2|       2| 25|    1|    2|    0

## Generate score using predict

In [36]:
#Call PREDICT using user defined function (UDF)
df.withColumn("PREDICT",model.udf(lit("output"),*df.columns)).show()

StatementMeta(sparkezi1n72, 82, 36, Finished, Available)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|PREDICT|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|   640000|  1|        1|       2| 36|    0|    0|    0|    0|    0|    0|   192325|    68891|    71564|    32968|    37541|    42064|    4000|    4000|    3000|    5000|    5000|    3000|      0|
|   100000|  1|        2|       2| 44|    0|    0|    0|   -1|   -1|    0|    20439|     6882|     2456|     1537|     1642|      110|    1068|    1007|    1741|    1646|     100|       0|      0|
|    70000|  2|